In [1]:
import pandas as pd
import numpy as np

CONFIG = {
    "widetable_path": r'D:\workspace\xiaoyao\data\widetable.parquet',
    "backtest_path": r'./reverse_strategy_backtest.csv',
    # 反向选股条件
    "auction_ratio_threshold": 0.8,  # 竞价量比≤0.8（资金冷淡）
    "industry_cold_rank": 0.5,       # 行业热度后50%
    "3d_loss_threshold": -7.0,       # 放宽至前3日跌幅≥7%（增加信号量）
    # 反向交易规则
    "buy_down_ratio": 0.02,          # 跌到前5日低点2%内（下方2%）
    "stop_profit": 3.6,              # 止盈3.6%（确保盈亏比≥1.8）
    "stop_loss": -2.0,               # 止损-2%
    "initial_fund": 100000.0
}

def load_data():
    df = pd.read_parquet(CONFIG["widetable_path"])
    df["date"] = pd.to_datetime(df["date"])  # 宽表date字段为交易日
    df = df[df["paused"] == 0.0].copy()  # 过滤停牌股（paused=0为正常交易）
    
    # 计算反向所需因子（对齐宽表字段）
    df["auction_vol_5d_mean"] = df.groupby("stock_code")["auc_volume"].transform(
        lambda x: x.rolling(5, min_periods=3).mean().shift(1).replace(0, 0.0001)
    )
    df["auc_volume_ratio"] = df["auc_volume"] / df["auction_vol_5d_mean"]  # 竞价量比
    
    df["daily_return"] = (df["close"] / df["pre_close"] - 1) * 100  # 基于pre_close计算涨跌幅
    df["3d_return"] = df.groupby("stock_code")["daily_return"].transform(
        lambda x: x.rolling(3).sum().shift(1)  # 前3日累计跌幅
    )
    
    df["ma5"] = df.groupby("stock_code")["close"].transform(lambda x: x.rolling(5).mean())
    df["ma20"] = df.groupby("stock_code")["close"].transform(lambda x: x.rolling(20).mean())
    df["5d_low"] = df.groupby("stock_code")["low"].transform(
        lambda x: x.rolling(5).min().shift(1)  # 前5日低点（用low字段）
    )
    
    # 行业热度排名（用申万一级行业）
    industry_rank = df.groupby(["date", "sw_l1_industry_name"])["daily_return"].mean().reset_index()
    industry_rank["industry_percentile"] = industry_rank.groupby("date")["daily_return"].rank(pct=True)
    df = df.merge(industry_rank[["date", "sw_l1_industry_name", "industry_percentile"]], 
                  on=["date", "sw_l1_industry_name"], how="left")
    
    # 生成全局交易日列表（用于日期跳转）
    global TRADING_DAYS
    TRADING_DAYS = sorted(df["date"].unique())
    return df

def get_next_trading_day(current_date):
    """获取下一个有效交易日（基于宽表date字段）"""
    idx = TRADING_DAYS.index(current_date)
    if idx + 1 < len(TRADING_DAYS):
        return TRADING_DAYS[idx + 1]
    return None  # 无下一个交易日

def select_reverse_stocks(df):
    all_selections = []
    for date in TRADING_DAYS:  # 仅遍历有效交易日
        daily_df = df[df["date"] == date].copy()
        # 反向选股条件（严格对齐宽表字段）
        daily_df = daily_df[
            (daily_df["auc_volume_ratio"] <= CONFIG["auction_ratio_threshold"]) &
            (daily_df["ma5"] < daily_df["ma20"]) &  # 均线空头
            (daily_df["3d_return"] <= CONFIG["3d_loss_threshold"]) &  # 超跌
            (daily_df["industry_percentile"] <= CONFIG["industry_cold_rank"]) &  # 非热点
            (daily_df["daily_return"].shift(1) >= -1.0)  # 前1日跌幅收窄
        ].copy()
        if len(daily_df) == 0:
            continue
        # 选跌幅最大的前5只（超跌得分）
        daily_df["oversold_score"] = daily_df["3d_return"].rank(ascending=True)
        selected = daily_df.nsmallest(5, "oversold_score")
        selected["selection_date"] = date
        all_selections.append(selected[["selection_date", "stock_code", "5d_low"]])
    return pd.concat(all_selections, ignore_index=True)

def backtest_reverse(selections, df):
    # 构建价格映射（包含high/low用于盘中触发判断）
    price_map = df.set_index(["date", "stock_code"])[["open", "close", "low", "high"]].to_dict("index")
    fund = CONFIG["initial_fund"]
    records = []
    
    for _, row in selections.iterrows():
        stock = row["stock_code"]
        t_date = row["selection_date"]
        
        # 获取T+1和T+2交易日（修正非交易日问题）
        t1_date = get_next_trading_day(t_date)
        t2_date = get_next_trading_day(t1_date) if t1_date else None
        if not t1_date or not t2_date:
            continue  # 缺少后续交易日，跳过
        
        # T+1日买入逻辑（修正买入价计算）
        t1_data = price_map.get((t1_date, stock))
        if not t1_data:
            continue
        t1_low = t1_data["low"]
        t1_open = t1_data["open"]
        # 目标买入价：前5日低点下方2%内（正确逻辑）
        target_buy_price = row["5d_low"] * (1 - CONFIG["buy_down_ratio"])
        # 判断是否达到买入条件
        if t1_low <= target_buy_price <= t1_open:
            buy_price = target_buy_price
        elif t1_low <= target_buy_price:  # 价格跌破目标价，按最低价买入
            buy_price = t1_low
        else:
            continue  # 未跌到目标价，放弃
        
        # 计算仓位（单票20%）
        shares = int((fund * 0.2) // (buy_price * 100)) * 100  # 100股整数倍
        if shares <= 0:
            continue  # 资金不足，跳过
        
        # T+2日卖出逻辑（用high/low判断盘中触发）
        t2_data = price_map.get((t2_date, stock))
        if not t2_data:
            continue
        t2_high = t2_data["high"]
        t2_low = t2_data["low"]
        t2_close = t2_data["close"]
        
        # 计算止盈止损触发价
        take_profit_price = buy_price * (1 + CONFIG["stop_profit"] / 100)
        stop_loss_price = buy_price * (1 + CONFIG["stop_loss"] / 100)
        
        # 判断触发类型
        if t2_high >= take_profit_price:
            sell_price = take_profit_price
            trigger_type = "止盈触发"
        elif t2_low <= stop_loss_price:
            sell_price = stop_loss_price
            trigger_type = "止损触发"
        else:
            sell_price = t2_close
            trigger_type = "收盘价卖出"
        
        # 计算收益
        return_rate = (sell_price / buy_price - 1) * 100
        profit = (sell_price - buy_price) * shares
        fund += profit
        
        # 记录详细信息（用于验证）
        records.append({
            "stock_code": stock,
            "buy_date": t1_date,
            "sell_date": t2_date,
            "buy_price": buy_price,
            "sell_price": sell_price,
            "return_rate": return_rate,
            "trigger_type": trigger_type,
            "fund_after": fund
        })
    
    # 计算胜率和盈亏比（严格统计有效交易）
    backtest_df = pd.DataFrame(records)
    valid_returns = backtest_df["return_rate"].dropna()
    if len(valid_returns) == 0:
        print("无有效交易记录")
        return backtest_df
    
    # 胜率 = 正收益交易数 / 总交易数
    win_count = len(valid_returns[valid_returns > 0])
    win_rate = win_count / len(valid_returns) * 100
    
    # 盈亏比 = 平均盈利 / 平均亏损（取绝对值）
    profit_returns = valid_returns[valid_returns > 0]
    loss_returns = valid_returns[valid_returns <= 0]
    avg_profit = profit_returns.mean() if len(profit_returns) > 0 else 0
    avg_loss = abs(loss_returns.mean()) if len(loss_returns) > 0 else 0
    profit_loss_ratio = avg_profit / avg_loss if avg_loss != 0 else 0
    
    # 输出关键指标
    print(f"初始资金：{CONFIG['initial_fund']:.2f} → 最终资金：{fund:.2f}")
    print(f"总交易次数：{len(valid_returns)} | 盈利次数：{win_count} | 亏损次数：{len(loss_returns)}")
    print(f"胜率：{win_rate:.2f}% | 平均盈利：{avg_profit:.2f}% | 平均亏损：{avg_loss:.2f}% | 盈亏比：{profit_loss_ratio:.2f}")
    
    backtest_df.to_csv(CONFIG["backtest_path"], index=False)
    return backtest_df

if __name__ == "__main__":
    TRADING_DAYS = []  # 全局交易日列表
    df = load_data()
    selections = select_reverse_stocks(df)
    backtest_result = backtest_reverse(selections, df)

初始资金：100000.00 → 最终资金：508123.70
总交易次数：1150 | 盈利次数：620 | 亏损次数：530
胜率：53.91% | 平均盈利：3.00% | 平均亏损：1.84% | 盈亏比：1.63
